In [1]:
import pandas as pd
dataset=pd.read_csv('zeolite_Amorphous.csv', index_col=0, header=0)
x=pd.concat([dataset.iloc[:, 0:5], dataset.iloc[:, 6:8]], axis=1)
y=dataset.iloc[:, 8]
from sklearn.model_selection import train_test_split
import numpy as np
rs=np.arange(1, 101, 1)
splits=[]

for i in rs:
    x_train, x_test, y_train, y_test=train_test_split(x, y, test_size=0.25, shuffle=True, stratify=y, random_state=i)
    splits.append((i, x_train, x_test, y_train, y_test))

from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_predict
import numpy as np
fold=StratifiedKFold(n_splits=3, shuffle=True, random_state=99)
max_depths=np.arange(1, 21, 1)
results=[]

for i, x_train, x_test, y_train, y_test in splits:
    accuracy_in_cv_all=[]
    for max_depth in max_depths:
        model=DecisionTreeClassifier(max_depth=max_depth, min_samples_leaf=3)
        estimated_y_in_cv=cross_val_predict(model, x_train, y_train, cv=fold)
        accuracy_in_cv_all.append(metrics.accuracy_score(y_train, estimated_y_in_cv))
    
    optimal_depth=max_depths[accuracy_in_cv_all.index(max(accuracy_in_cv_all))]   

    model=DecisionTreeClassifier(max_depth=optimal_depth, min_samples_leaf=3)
    model.fit(x_train, y_train)
    estimated_y_test=pd.DataFrame(model.predict(x_test))
    accuracy_test=metrics.accuracy_score(y_test, estimated_y_test)
    results.append({"random_state":i, "accuracy":accuracy_test})
    df_results=pd.DataFrame(results)
    df_results

In [2]:
df_results

,random_state,accuracy
0,1,0.666667
1,2,0.666667
2,3,0.709402
3,4,0.675214
4,5,0.700855
...,...,...
95,96,0.666667
96,97,0.675214
97,98,0.683761
98,99,0.717949


In [3]:
df_results.to_csv('accuracy_dt_0.25.csv')